# Interaction with the contract

### Web3 instanciation and setup

**NOTE:** Before running this, ensure the blockchain is up by running `deploy.sh`.  

In [4]:
from dotenv import dotenv_values
from functools import wraps
from web3 import Web3
import json

url = "http://127.0.0.1:8545"

# Here, put the output from the deploy
contract_address = "0xcf94DD98F88B4975C3bb7074A213140D985FFe57"


env = dotenv_values(".env")
password = env["ACCOUNT_PASSWORD"]

# Also output from deploy
funder_addresses = [
    "0x548AEC5436F39e202205A2f829eeC592E12E7289",
    "0x87834f4A4c629745603e0DC338271246f1798c83",
]
DEFAULT_AMOUNT_FUND = 0.5 # ETH
UNLOCK_DURATION = 300 # Seconds

w3 = Web3(Web3.HTTPProvider(url))

In [5]:
def fund_new_wallet_decorator(func):
    """Wrapper function to automatically send funds from funder adresses to newly created accounts."""

    @wraps(func)
    def wrapper(*args, **kwargs):
        new_wallet_address, new_wallet_private_key = func(*args, **kwargs)

        # Unlock pre-funder wallets
        for address in funder_addresses:
            w3.geth.personal.unlock_account(
                address, password, UNLOCK_DURATION
            )  # Unlock for 5 minutes

        gas_price = w3.eth.gas_price
        gas_limit = w3.eth.estimate_gas(
            {
                "to": new_wallet_address,
                "value": w3.to_wei(DEFAULT_AMOUNT_FUND, "ether"),
                "from": funder_addresses[0],
            }
        )

        for address in funder_addresses:
            transaction = {
                "to": new_wallet_address,
                "value": w3.to_wei(DEFAULT_AMOUNT_FUND, "ether"),
                "gas": gas_limit,
                "gasPrice": gas_price,
                "from": address,
            }
            txn_hash = w3.eth.send_transaction(transaction)
            receipt = w3.eth.wait_for_transaction_receipt(txn_hash)

        return new_wallet_address, new_wallet_private_key

    return wrapper

### Registering an account in the Blockchain

In [6]:
@fund_new_wallet_decorator
def create_new_wallet(password: str):
    """Creates a new wallet in the blockchain"""
    new_wallet = w3.eth.account.create(password)
    new_wallet_address = new_wallet.address
    new_wallet_private_key = new_wallet._private_key.hex()

    return new_wallet_address, new_wallet_private_key


# Usage Example
new_wallet_address, new_wallet_private_key = create_new_wallet("password")

print(f"New Wallet Address: {new_wallet_address}")
print(f"New Wallet Private Key: {new_wallet_private_key}")
print(f"New Wallet Balance: {w3.eth.get_balance(new_wallet_address)}")

New Wallet Address: 0xC7C11c1684bC537a9dF5C9aEcc019Ee72468cEf1
New Wallet Private Key: 0x5c7b18179273cfca99cb2a5e18dfde6ac6b5ab6542df717aabdd7aa51c2d4119
New Wallet Balance: 1000000000000000000


### Interacting with the contract

In [7]:
with open("./build/contracts_Voting_sol_Voting.abi", "r") as f:
    contract_abi = json.load(f)
contract = w3.eth.contract(address=contract_address, abi=contract_abi)

# Election info

election_id = 0
election_name = "Eleição Municipal de Morrinhos"
description = "Eleição oficial para prefeitos da melhor cidade da região"

# Creating an Election

tx = contract.functions.createElection(election_name, description).build_transaction(
    {
        "from": new_wallet_address,
        "gas": 2000000,
        "gasPrice": w3.to_wei("20", "gwei"),
        "nonce": w3.eth.get_transaction_count(new_wallet_address),
    }
)

signed_tx = w3.eth.account.sign_transaction(tx, private_key=new_wallet_private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# Adding a Voter

tx = contract.functions.addVoter(
    0, "Leonardo", "12345", new_wallet_address
).build_transaction(
    {
        "from": new_wallet_address,
        "gas": 2000000,
        "gasPrice": w3.to_wei("20", "gwei"),
        "nonce": w3.eth.get_transaction_count(new_wallet_address),
    }
)

signed_tx = w3.eth.account.sign_transaction(tx, private_key=new_wallet_private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [8]:
# Getting all elections from the Admin
contract.functions.getAdminElections(new_wallet_address).call()

[(0,
  'Eleição Municipal de Morrinhos',
  'Eleição oficial para prefeitos da melhor cidade da região',
  '0xC7C11c1684bC537a9dF5C9aEcc019Ee72468cEf1',
  '0x0000000000000000000000000000000000000000',
  0,
  9999999999,
  False,
  1722527728,
  1722527728),
 (0,
  '',
  '',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000',
  0,
  0,
  False,
  0,
  0)]

In [9]:
# Ensuring the election is created
contract.functions.getElection(0).call()

(0,
 'Eleição Municipal de Morrinhos',
 'Eleição oficial para prefeitos da melhor cidade da região',
 '0xC7C11c1684bC537a9dF5C9aEcc019Ee72468cEf1',
 '0x0000000000000000000000000000000000000000',
 0,
 9999999999,
 False,
 1722527728,
 1722527728)

In [10]:
# Getting all Voters from an Election
contract.functions.getElectionVoter(0, 0).call()

(0,
 'Leonardo',
 '0xC7C11c1684bC537a9dF5C9aEcc019Ee72468cEf1',
 b'\x18A\xd6S\xf9\xc4\xed\xda\x9df\xa7\xe7s{9v=k\xd4\x0fV\x9a>\xc6\x85\x9d3\x05\xb7#\x10\xe6',
 1722527732,
 1722527732)